<a href="https://colab.research.google.com/github/DreRnc/ExplainingExplanations/blob/main/Explanations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Dataset : **E-SNLI**. \
Model : **Small T5**.

In [ ]:
colab = True

In [ ]:
if colab:
    !git clone https://github.com/DreRnc/ExplainingExplanations.git
    %cd ExplainingExplanations
    %pip install -r requirements.txt
    !git checkout seq2seq

# 1.0 Preparation


## 1.1 Loading Dataset

In [ ]:
from datasets import load_dataset

dataset = load_dataset("esnli", download_mode="FORCE_REDOWNLOAD")

In [ ]:
training_set = dataset["train"]
validation_set = dataset["validation"]
test_set = dataset["test"]

print("Shape of training_set: ", training_set.shape)
print("Shae of validation_set: ", validation_set.shape)
print("Shape of test_set: ", test_set.shape)

In [ ]:
training_set[0]

In [ ]:
n_train = 10000
n_valid = n_test = 10000

train_small = training_set.select(range(n_train))
valid_small = validation_set.select(range(n_valid))
test_small = test_set.select(range(n_test))

print("Shape of train_small: ", train_small.shape)
print("Shape of valid_small: ", valid_small.shape)
print("Shape of test_small: ", test_small.shape)

## 1.2 Loading T5 Model

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("t5-small")

Test **zero-shot** on a random task.

In [ ]:
input_ids = tokenizer(
    "translate English to French: Hello Dre, I think the English version is ok for us.",
    return_tensors="pt",
).input_ids
outputs = model.generate(input_ids, max_length = 20)

print(tokenizer.decode(outputs[0], skip_special_tokens=True, max_length=20))

## 1.3 Zero-shot example to Verify Everything is Working

In [ ]:
from src.utils import generate_prompt_mnli

In [ ]:
example = training_set[0]
example

Generating the prompt:

<b><u> mnli hypothesis: </b></u> The St. Louis Cardinals have always won. <b><u> premise: </b></u> yeah well losing is i mean i’m i’m originally from Saint Louis and Saint Louis Cardinals when they were there were uh a mostly a losing team but

Output:
* 0: Entailment
* 1: Neutral
* 2: Contradiction

In [ ]:
prompt = generate_prompt_mnli(example)
prompt

In [ ]:
input_ids = tokenizer(prompt, return_tensors="pt").input_ids

outputs = model.generate(input_ids)
print(outputs)
print('Shape of outputs:', outputs.shape)
print('Shape of outputs[0]:', outputs[0].shape)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

## 1.4  Tokenize the dataset

In [ ]:
train_small.info

In [ ]:
train_small.features

In [ ]:
from functools import partial
from src.utils import tokenize_function
import time

In [ ]:
tokenize_mapping = partial(tokenize_function, tokenizer=tokenizer)

In [ ]:
time_init = time.time()
train_small_tokenized = train_small.map(tokenize_mapping, batched=True).with_format("torch")
valid_small_tokenized = valid_small.map(tokenize_mapping, batched=True).with_format("torch")
test_small_tokenized = test_small.map(tokenize_mapping, batched=True).with_format("torch")
time_end = time.time()

print("Time taken to tokenize: ", time_end - time_init)
print("Shape of train_small_tokenized: ", train_small_tokenized.shape)
print("Shape of valid_small_tokenized: ", valid_small_tokenized.shape)
print("Shape of test_small_tokenized: ", test_small_tokenized.shape)

In [ ]:
train_small_tokenized = train_small_tokenized.remove_columns(['label'])
valid_small_tokenized = valid_small_tokenized.remove_columns(['label'])
test_small_tokenized = test_small_tokenized.remove_columns(['label'])

In [ ]:
train_small_tokenized.features

# 2.0 Tasks

In [ ]:
import torch
import evaluate
from src.utils import compute_metrics, eval_pred_transform_accuracy
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer, T5ForConditionalGeneration, DataCollatorForSeq2Seq


In [ ]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

model.to(device)
device

In [ ]:
transform_accuracy = partial(eval_pred_transform_accuracy, tokenizer = tokenizer)
compute_accuracy = partial(compute_metrics, pred_transform=transform_accuracy, metric = evaluate.load('accuracy'))

## 2.1 Task 1: Zero-shot evaluation

In [ ]:
model = T5ForConditionalGeneration.from_pretrained("t5-small")
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="task1",
    predict_with_generate=True,
    per_device_eval_batch_size=16,
    generation_max_length=32,
)

In [ ]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_small_tokenized,
    eval_dataset=valid_small_tokenized,
    compute_metrics=compute_accuracy,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [ ]:
trainer.evaluate(test_small_tokenized)

## 2.2 Task 2: Fine tuning without explanations

In [ ]:
model_ft = T5ForConditionalGeneration.from_pretrained("t5-small")
data_collator_ft = DataCollatorForSeq2Seq(tokenizer, model=model_ft)

In [ ]:
training_args_ft = Seq2SeqTrainingArguments(
    output_dir="task2",
    evaluation_strategy="epoch",
    num_train_epochs=10,
    predict_with_generate=True,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    generation_max_length=32,
)

In [ ]:
trainer_ft = Seq2SeqTrainer(
    model=model_ft,
    args=training_args_ft,
    train_dataset=train_small_tokenized,
    eval_dataset=valid_small_tokenized,
    compute_metrics=compute_accuracy,
    data_collator=data_collator_ft,
    tokenizer=tokenizer,
)

In [ ]:
trainer.evaluate(train_small_tokenized)

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate(test_small_tokenized)

## 2.3 Task 3: Making the model generate explanations

We need to give as labels the label and the explanation tokenized.

### Preparing the dataset with labelled explanations

In [ ]:
dataset_explanations = load_dataset("explanations", download_mode="FORCE_REDOWNLOAD")

In [ ]:
training_set_ex = dataset_explanations["train"]
validation_set_ex = dataset_explanations["validation"]
test_set_ex = dataset["test"]

print("Shape of training_set: ", training_set_ex.shape)
print("Shae of validation_set: ", validation_set_ex.shape)
print("Shape of test_set: ", test_set_ex.shape)

In [ ]:
n_train = 10000
n_valid = n_test = 10000

train_small_ex = training_set_ex.select(range(n_train))
valid_small_ex = validation_set_ex.select(range(n_valid))
test_small_ex = test_set_ex.select(range(n_test))

print("Shape of train_small: ", train_small_ex.shape)
print("Shape of valid_small: ", valid_small_ex.shape)
print("Shape of test_small: ", test_small_ex.shape)

#### Tokenizing the dataset

In [ ]:
from functools import partial
from src.utils import tokenize_function_ex
import time

In [ ]:
tokenize_mapping_ex = partial(tokenize_function_ex, tokenizer=tokenizer)

In [ ]:
train_small_tokenized_ex = train_small.map(tokenize_mapping_ex, batched=True).with_format("torch")
valid_small_tokenized_ex = valid_small.map(tokenize_mapping_ex, batched=True).with_format("torch")
test_small_tokenized_ex = test_small.map(tokenize_mapping_ex, batched=True).with_format("torch")

print("Shape of train_small_tokenized: ", train_small_tokenized_ex.shape)
print("Shape of valid_small_tokenized: ", valid_small_tokenized_ex.shape)
print("Shape of test_small_tokenized: ", test_small_tokenized_ex.shape)

In [ ]:
train_small_tokenized_ex = train_small_tokenized_ex.remove_columns(['label'])
valid_small_tokenized_ex = valid_small_tokenized_ex.remove_columns(['label'])
test_small_tokenized_ex = test_small_tokenized_ex.remove_columns(['label'])

In [ ]:
train_small_tokenized_ex.features

### Fine Tuning

In [ ]:
model_ft_ex = T5ForConditionalGeneration.from_pretrained("t5-small")
data_collator_ft_ex = DataCollatorForSeq2Seq(tokenizer, model=model_ft_ex)

In [ ]:
training_args_ft_ex = Seq2SeqTrainingArguments(
    output_dir="task3",
    evaluation_strategy="epoch",
    num_train_epochs=5,
    predict_with_generate=True,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    generation_max_length=128,
)

In [ ]:
trainer_ft_ex = Seq2SeqTrainer(
    model=model_ft_ex,
    args=training_args_ft_ex,
    train_dataset=train_small_tokenized,
    eval_dataset=valid_small_tokenized,
    # TODO: compute_metrics= COMPUTE METRICS FOR EXPLANATION (ROUGE?)
    data_collator=data_collator_ft_ex,
    tokenizer=tokenizer,
)

In [ ]:
trainer.evaluate(train_small_tokenized_ex)

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate(test_small_tokenized_ex)